Import modules:

In [1]:
import pandas as pd

Load tops data:

In [2]:
df = pd.read_csv('../data/oga_markers_all_wells.csv')

In [3]:
df.head()

,Unnamed: 0,OBJECTID,XY,Well_Number,Unique_ID,Source_of_Well,Provider,Depth_MD,Depth_TVDSS,Pick_Name,OGA_Epoch
0,0,1,"(-3.269956, 58.263699)",11/24- 1,11/24- 1_Lloyds Register_Kimmeridge Clay,UK Offshore,Lloyds Register,75.0,NaN,Kimmeridge Clay,NaN
1,1,2,"(-3.269956, 58.263699)",11/24- 1,11/24- 1_Lloyds Register_Heather,UK Offshore,Lloyds Register,1140.0,NaN,Heather,NaN
2,2,3,"(-3.269956, 58.263699)",11/24- 1,11/24- 1_Lloyds Register_Beatrice,UK Offshore,Lloyds Register,1486.0,NaN,Beatrice,NaN
3,3,4,"(-3.269956, 58.263699)",11/24- 1,11/24- 1_Lloyds Register_Brora Coal,UK Offshore,Lloyds Register,1513.0,NaN,Brora Coal,NaN
4,4,5,"(-3.269956, 58.263699)",11/24- 1,11/24- 1_Lloyds Register_Orrin,UK Offshore,Lloyds Register,1648.0,NaN,Orrin,NaN


Not all of the columns are needed.  Extract the useful columns, convert the location data (XY) column into more useable 'x' and 'y' columns (they are currently string data).

In [4]:
df = df[['Well_Number','XY','Depth_MD','Pick_Name']]

In [5]:
df[['x','y']] = df.XY.str.split(',',expand=True)

In [6]:
df.x = df.x.str.replace('(','')

In [7]:
df.y = df.y.str.replace(')','')

In [8]:
df = df.drop('XY',axis=1)

The cleaned data set now looks like this:

In [9]:
df.tail()

,Well_Number,Depth_MD,Pick_Name,x,y
159378,219/27- 1,2580.0,Shetland,0.305167,62.12047
159379,219/28- 1,2380.0,Shetland,0.51675,62.03792
159380,219/28- 2,2655.0,Shetland,0.465306,62.12742
159381,220/26- 1,2225.0,Shetland,1.032944,62.07294
159382,220/26- 2,2017.0,Shetland,1.045444,62.023


The data set contains nearly 160,000 tops.  We want to know which wells have Paleocene picks. This list will then be used in QGIS to select a set of wells from a limited geographic area to use to train a model to try to identify lithostratigraphic units within the Paleocene.

In [10]:
#list of 'common' Paleocene tops:
pal_tops = ['Forties','Beauly','Balder','Odin','Dornoch','Lista','Maureen','Vale','Andrew','Glamis','Fergie','Sele','Fiskebank']

In [11]:
#extract all tops that are in the list of common Paleocene tops and save to a csv file for use in QGIS:
df.loc[df['Pick_Name'].isin(pal_tops)].to_csv('../data/oga_markers_paleo_all_wells.csv')

---
---
Using the extracted list above and QGIS a subset of wells with available log data was selected.  The list of wells can now be imported.

---
---

In [12]:
wells = pd.read_csv('../data/oga_wells_with_logs_selected.csv')

In [13]:
#extract a list of well names from the selected Paleocene wells:
well_names_list = list(wells.WELLREGNO)
well_names_list

['15/11- 1',
 '15/11b- 5',
 '15/12- 1',
 '15/13- 2',
 '15/13- 3',
 '15/13a- 10',
 '15/13a- 4Z',
 '15/13a- 7Z',
 '15/16- 5',
 '14/20b- 30',
 '14/20b- 32',
 '15/16b- 18',
 '15/16b- 20',
 '15/17- 10',
 '15/17- 15',
 '15/17- 21',
 '15/17- 24',
 '15/17- 25',
 '15/17- 4',
 '15/17- 6',
 '15/17- 7',
 '15/17- 8A',
 '15/17- 9',
 '15/18- 1',
 '14/20- 1',
 '14/20- 10',
 '14/20- 11',
 '14/20- 12',
 '14/20- 13',
 '14/20- 14',
 '14/20- 2',
 '14/20- 5',
 '14/20- 7',
 '14/20- 8',
 '14/20- 9',
 '14/20b- 29',
 '15/21- 1',
 '15/18- 2',
 '15/18b- 11',
 '15/19- 1',
 '15/19- 5',
 '15/19- 9',
 '15/19b- 10',
 '15/21a- 51',
 '15/21a- 52',
 '15/21a- 55',
 '15/21b- 21',
 '15/21b- 45',
 '15/21b- 47',
 '15/21b- 49',
 '15/21b- 50',
 '15/23d- 13',
 '15/24a- 4',
 '15/21b- 56',
 '15/22- 14',
 '15/19- 2']

In [14]:
# extract all tops that are in the list of selected wells:
selected_well_tops = df.loc[df['Well_Number'].isin(well_names_list)]

In [15]:
# save to a csv file:
selected_well_tops.to_csv('../data/oga_markers_selected_wells_initial.csv')

In [16]:
selected_well_tops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1931 entries, 1978 to 154111
Data columns (total 5 columns):
Well_Number    1931 non-null object
Depth_MD       1895 non-null float64
Pick_Name      1931 non-null object
x              1931 non-null object
y              1931 non-null object
dtypes: float64(1), object(4)
memory usage: 90.5+ KB


---
---
We now have a list of wells that have available log data and tops picked for the Paleocene section.  The final stage of data preparation is a list of the selected wells and the name of the las file with the log data for that well.

---
---

To do this we can import a list of well names and las file names for the wider area and filter it based on the selected Paleocene wells.

In [17]:
las_names = pd.read_csv('../data/wells_by_lasfile.csv',header=None)

In [18]:
#assign column names to the imported data:
las_names.rename(columns={0:'fname',1:'wname'},inplace=True)

In [19]:
#the new file has well names extracted directly from the las files - they omit a space from the file name:
las_names.wname = las_names.wname.str.replace('-','- ')

In [20]:
#join the two lists:
las_files = las_names.loc[las_names['wname'].isin(well_names_list)]
len(las_files)

43

The number of wells has been reduced to 43.  This is likely to be down to well name matching.  It still leaves plenty of wells to use to train a model (hopefully).

In [21]:
#save the final list of wells to a csv file:
las_files.to_csv('../data/wells_by_lasfile_edit.csv')